In [ ]:
import pandas as pd
import georasters as gr
import os
import geopandas as gpd
from shapely.geometry import Point
import urllib.request
import json
import plotly.express as px
import urllib.request
import json
from shapely.geometry import Polygon
import numpy as np
import seaborn as sns

Get data

Social Vulnerability, or “SVI”, measures social conditions of a community and its vulnerability to
human suffering and financial loss in the event of a disaster. It is primarily used by public
officials and emergency response planners to identify communities in need of support. SVI can
be grouped into four themes: 1) socioeconomic status 2) household characteristics 3) racial and
ethnic minority status 4) housing type and transportation. These themes are composed of
separate variables that are used to compute the theme value. There are 16 total individual
variables; they include persons below 150% poverty, persons unemployed, aged 65 & older,
civilians with a disability, etc. The four themes were used in this project and can be broken
down as follows: socioeconomic status – describes income, poverty, education and
employment; household composition – describes age, disability, and single parenting; minority
status – describes race and ethnicity; housing type – describes housing structure, vehicle access
and crowding. Additionally, the census variables: total population for a tract, housing units
within a tract, number of households within a tract, and unemployment rate were also
considered in this project. 

Data Source: 
https://www.atsdr.cdc.gov/placeandhealth/svi/data_documentation_download.html


In [ ]:
svi_2014=pd.read_csv("data\SVI_2014_US.csv")
svi_2014['Date']=2014

svi_2016=pd.read_csv("data\SVI_2016_US.csv")
svi_2016['Date']=2016

svi_2018=pd.read_csv("data\SVI_2018_US.csv")
svi_2018['Date']=2018

svi_2020=pd.read_csv("data\SVI_2020_US.csv")
svi_2020['Date']=2020

SVI data cleaning

In [ ]:
match_fields=set(svi_2014.columns) & set(svi_2016.columns) & set(svi_2018.columns) & set(svi_2020.columns)
match_fields=list(match_fields)

svi_2014=svi_2014[match_fields]
svi_2016=svi_2016[match_fields]
svi_2018=svi_2018[match_fields]
svi_2020=svi_2020[match_fields]

svi = pd.concat([svi_2014, svi_2016,svi_2018,svi_2020])
svi.shape

In [ ]:
svi['STCNTY'] = svi['STCNTY'].astype(int).astype(str).apply(lambda x: '{0:0>5}'.format(x))
svi['Date'] = pd.to_datetime(svi['Date'].map(str)).dt.strftime('%Y')
svi.sort_values(by='Date', inplace=True)

In [ ]:
svi=svi.drop(['STATE','LOCATION','COUNTY','ST_ABBR'], axis=1)
svi=svi.groupby(['STCNTY','Date'],as_index=False).mean()
display(svi.head(5))

Data processing was initially performed using CO2 data obtained from NASA’s OCO-2 mission
and via NASA’s Goddard Earth Sciences Data and Information Services Center (Goddard Earth
Sciences Data and Information Services Center, 2018). This dataset consists of 3 km2
rectangular raster bands. OCO-2 data possesses significant discontinuity due to the satellite’s
path, poor quality flagging, and sparce observations with irregular frequencies. A given location
can have between 2 and 50 high-quality observations per year. Not enough values were present
in the OCO-2 dataset alone to conduct imputation of missing values. In addition to the
discontinuity of the observations, even with projection and interpolation, the raster bands were
difficult to spatially join with other geodata.

Instead, we harnessed the work of researchers, Sheng et al. (2022), who produced a more
complete CO2 emissions dataset. Using geostatistical techniques, they combined the OCO-2 and
GOSAT missions producing a complete global emission dataset with a 1° square pixel resolution
at both 3-day and monthly frequencies from 2009-2020 9
. Our work aggregates the monthly
values into annual means and standard deviations which removes the seasonal trend present in
any CO2 data (visible in Figure 4) and captures variation. Each pixel was interpolated and
labeled with a with a latitude and longitude coordinate pair.

Data Source: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/4WDTD8

Sample output from one month of readings

In [ ]:
import rasterio
from rasterio.plot import show
fp = r'data\MappingXCO2_month\MappingXCO2_201002.tif'
img = rasterio.open(fp)
show(img)

XCO2 data cleaning

In [ ]:
xco2_data=pd.DataFrame()
directory = os.fsencode('data\MappingXCO2_month')
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    year=filename[12:16]
    month=filename[16:18]
    data=gr.from_file(f'data\MappingXCO2_month\{filename}').to_pandas().iloc[:,[2,3,4]]
    data = data.rename(columns={True: 'xco2', 'x': 'lon', 'y':'lat'})
    data['year']=year
    data['month']=month
    xco2_data = pd.concat([xco2_data, data])
    
xco2_data = xco2_data[(xco2_data['lon'] >= -180.00) & (xco2_data['lon'] <= -65.00)]
xco2_data = xco2_data[(xco2_data['lat'] >= 17.00) & (xco2_data['lat'] <= 72.00)]
xco2_data['coords']=list(zip(xco2_data['lon'],xco2_data['lat']))
xco2_data['coords']=xco2_data['coords'].apply(Point)

display(xco2_data.head())

Grouping XCO2 measurements by united states county. 

County boundaries obtained from: https://github.com/plotly/datasets
 
Not really sure what this one is:  
https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?disjunctive.statefp&disjunctive.countyfp&disjunctive.name&disjunctive.namelsad&disjunctive.stusab&disjunctive.state_name

In [ ]:
request = urllib.request.Request('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')
opener = urllib.request.build_opener()
response = opener.open(request)
counties_map = json.load(response)

counties=gpd.GeoDataFrame.from_features(counties_map["features"])
id = pd.json_normalize(counties_map, record_path =['features'])[['id']]
counties['geoid']=id

points = gpd.GeoDataFrame(xco2_data, geometry='coords', crs=counties.crs)
xco2_mo = gpd.sjoin_nearest(counties,points, how='left')

xco2_mo=xco2_mo[['xco2','year','month','geoid']]
xco2_mo=xco2_mo.groupby(['geoid','year','month'],as_index=False).mean()
xco2_mo['date'] = pd.to_datetime(xco2_mo[['year', 'month']].assign(DAY=1))

display(xco2_mo.head())

In [ ]:
xco2_mo_plot=xco2_mo.groupby(['year','month'],as_index=False).mean()
xco2_mo_plot['date']=xco2_mo_plot['date'].dt.strftime('%Y-%m')

Plotting change average xco2 value in the United States over time

In [ ]:
fig = px.line(xco2_mo_plot, x="date", y="xco2",  labels={
    "date": "Date",
    "xco2": "Column Averaged CO2 (ppm)",
    "species": "Species of Iris"}, 
    title='United States Column Averaged CO2 Levels Over time')
fig.update_xaxes(
    dtick="M3")
fig.show()

In [ ]:
xco2_yr=xco2_mo.drop(['month','date'], axis=1)
xco2_yr=xco2_yr.groupby(['geoid','year'],as_index=False).mean()
display(xco2_yr.head(2))
print(xco2_yr.shape)

In [ ]:
svi_co2=pd.merge(svi, xco2_yr, left_on=['STCNTY','Date'], right_on=['geoid','year'])
svi_co2=svi_co2.dropna()
svi_co2.shape

For each year, subtract the mean

In [ ]:
#calculate yearly averages
grouped=svi_co2.groupby('year')
averaged=grouped.mean()
print(averaged['xco2'])

conditions = [
    (svi_co2['year'] == '2014'),
    (svi_co2['year'] == '2016'),
    (svi_co2['year'] == '2018'),
    (svi_co2['year'] == '2020')
    ]
values = [397.557046, 403.399761, 407.916993, 412.896790]

svi_co2['yr_avg'] = np.select(conditions, values)

svi_co2['adj_xco2']=svi_co2['xco2']-svi_co2['yr_avg']
display(svi_co2['adj_xco2'])

Pickling the dataset

In [ ]:
svi_co2.to_pickle('data\dataset')
test = pd.read_pickle('data\dataset')